# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [3]:
spaceship = spaceship.dropna()
spaceship["Cabin"] = spaceship['Cabin'].str[0]
spaceship = spaceship.drop(columns = ["PassengerId", "Name"])
spaceship = pd.get_dummies(spaceship, columns=['HomePlanet', 'Destination', "Cabin"])

features = spaceship.drop(columns=['Transported'])
target =  spaceship[["Transported"]]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)

# Normalize data
from sklearn.preprocessing import MinMaxScaler, StandardScaler

normalizer = StandardScaler()
normalizer.fit(X_train)

X_train_norm = normalizer.transform(X_train)
X_test_norm = normalizer.transform(X_test)

X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)
X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns)

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [4]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor
import sklearn.metrics
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100,
                               max_samples = 1000)

bagging_reg.fit(X_train_norm, y_train)

pred = bagging_reg.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", bagging_reg.score(X_test_norm, y_test))

c:\Users\xavie\anaconda3\Lib\site-packages\sklearn\ensemble\_bagging.py:505: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


MAE 0.2764650163116667
RMSE 0.37709442529651965
R2 score 0.43119917764115023


c:\Users\xavie\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


- Evaluate your model

In [5]:
print("With an R2 of 0.435, the score is very low")

With an R2 of 0.435, the score is very low


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Define the model
bagging_reg = BaggingRegressor(
    DecisionTreeRegressor(),
    n_estimators=100,
    max_samples=1000
)

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 150, 200, 300, 400],  # Number of trees in the ensemble
    'max_samples': [100, 500, 750, 1000, 2000]  # Number of samples used for each tree
}

- Run Grid Search

In [17]:
# Set up GridSearchCV
grid_search = GridSearchCV(estimator=bagging_reg, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit GridSearchCV to the data
grid_search.fit(X_train_norm, y_train)

# Get the best parameters and the best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Print the best parameters and best score
print(f"Best parameters found: {best_params}")
print(f"Best CV score (negative MSE): {best_score}")

# Evaluate the model on the test set with the best parameters
best_bagging_reg = grid_search.best_estimator_

# Make predictions
pred = best_bagging_reg.predict(X_test_norm)


c:\Users\xavie\anaconda3\Lib\site-packages\sklearn\ensemble\_bagging.py:505: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


Best parameters found: {'max_samples': 500, 'n_estimators': 400}
Best CV score (negative MSE): -0.13768462179969446


- Evaluate your model

In [16]:
# Print the evaluation metrics
print("MAE", mean_absolute_error(y_test, pred))
print("RMSE", mean_squared_error(y_test, pred, squared=False))
print("R2 score", best_bagging_reg.score(X_test_norm, y_test))

MAE 0.2825519730206758
RMSE 0.37490967369296224
R2 score 0.4377709462857463


c:\Users\xavie\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [18]:
print(f"the best parameters are {best_params}, with R2 score of 0.437, improvement from 435")

the best parameters are {'max_samples': 500, 'n_estimators': 400}, with R2 score of 0.437, improvement from 435
